In [10]:
import os
import cv2
import numpy as np
import json
from PIL import Image, ImageEnhance
import random

In [11]:
def convert_via_to_yolo(via_file_path, output_dir):
    with open(via_file_path, 'r') as via_file:
        via_data = json.load(via_file)

    images_output_dir = os.path.join(output_dir, 'images')
    labels_output_dir = os.path.join(output_dir, 'labels')
    os.makedirs(images_output_dir, exist_ok=True)
    os.makedirs(labels_output_dir, exist_ok=True)

    for image_id, image_data in via_data['_via_img_metadata'].items():
        image_path = os.path.join('E:\software\Task2\dataset\computer vision synthetic dataset\cat_dog', image_data['filename'])
        image_name = os.path.basename(image_path)
        image_name_without_ext = os.path.splitext(image_name)[0]

        image = Image.open(image_path)
        image_width, image_height = image.size

        annotations = []
        for region in image_data['regions']:
            shape_attributes = region['shape_attributes']
            x = shape_attributes['x']
            y = shape_attributes['y']
            width = shape_attributes['width']
            height = shape_attributes['height']

            # Normalize bounding box coordinates
            x_normalized = x / image_width
            y_normalized = y / image_height
            width_normalized = width / image_width
            height_normalized = height / image_height

            class_label = region['region_attributes'].get('name', '')
            yolo_annotation = f"{class_label} {x_normalized} {y_normalized} {width_normalized} {height_normalized}"
            annotations.append(yolo_annotation)

        save_label(yolo_annotation, labels_output_dir, image_name_without_ext)

        
        # Save original image
        image.save(os.path.join(images_output_dir, f"{image_name_without_ext}.jpg"))

        # Augment images with random brightness
        for i in range(1, 6):
            augmented_image = apply_brightness(image)
            augmented_image_name = f"{image_name_without_ext}_brightness_{i}.jpg"
            augmented_image.save(os.path.join(images_output_dir, augmented_image_name))
            augmented_image_name_without_ext = os.path.splitext(augmented_image_name)[0]
            save_label(yolo_annotation, labels_output_dir, augmented_image_name_without_ext)

        # Augment images with random noise
        for i in range(1, 6):
            augmented_image = apply_noise(image)
            augmented_image_name = f"{image_name_without_ext}_noise_{i}.jpg"
            augmented_image.save(os.path.join(images_output_dir, augmented_image_name))
            augmented_image_name_without_ext = os.path.splitext(augmented_image_name)[0]
            save_label(yolo_annotation, labels_output_dir, augmented_image_name_without_ext)

In [12]:
def apply_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    brightness_factor = random.uniform(0.7, 1.3)
    enhanced_image = enhancer.enhance(brightness_factor)
    return enhanced_image

def apply_noise(image):
    image = np.array(image)
    noise_factor = random.uniform(0.05, 0.15)
    noisy_image = image + noise_factor * np.random.randn(*image.shape)
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)

def save_label(yolo_annotation, labels_output_dir, image_name_without_ext):
    label_file_path = os.path.join(labels_output_dir, f"{image_name_without_ext}.txt")
    with open(label_file_path, 'w') as label_file:
        label_file.write(yolo_annotation)

In [13]:
json_file = 'cat_dog.json'
output_folder = 'E:\software\Task2\dataset\computer vision synthetic dataset\Result'

In [14]:
convert_via_to_yolo(json_file, output_folder)